# Documentation
* Resolution = 1.0
* aggregated sim matrix with 0.5 itp
* leiden clustering
* final avg times [2,4,6,8,10,12]
* seed = 42

# Import

In [1]:
import numpy as np
import json
from scipy.sparse import load_npz,save_npz,diags,csr_matrix
import scipy.sparse as sp
import pandas as pd
import os
from io import BytesIO
from tqdm import tqdm
from scipy.sparse.linalg import eigsh
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
from pathlib import Path
import igraph as ig
import leidenalg as la
from matplotlib.backends.backend_pdf import PdfPages
from pypdf import PdfReader, PdfWriter
from tempfile import NamedTemporaryFile
import networkx as nx
from sklearn.neighbors import NearestNeighbors
import community as community_louvain
import pickle
from collections import defaultdict
import gc
from pympler import muppy, asizeof
from sklearn.metrics import adjusted_rand_score
import shutil

# Building Multilayer Transition Matrix

In [2]:
# Choose the disease to analyze
DISEASE = "NONE"

In [3]:
# Load all the matrices needed
OUTPUT_DIRECTORY = f"../output/{DISEASE}/"
DGIDB_DIRECTORY = f"../../Gen_Hypergraph/output/DGIDB_{DISEASE}/"
MSIGDB_DIRECTORY = "../../Gen_Hypergraph/output/MSigDB_Full/"

## MSIGDB
MSIGDB_binary_matrix = load_npz(MSIGDB_DIRECTORY + "hypergraph_incidence_matrix_binary.npz")
MSIGDB_weighted_matrix = load_npz(MSIGDB_DIRECTORY + "hypergraph_incidence_matrix_weighted.npz")
MSIGDB_gene_weight_diag_matrix = load_npz(MSIGDB_DIRECTORY + "gene_weight_diag_matrix.npz")
MSIGDB_diag_node_degree_matrix = load_npz(MSIGDB_DIRECTORY + "diag_node_degree_matrix.npz")
MSIGDB_inverse_diag_edge_degree_matrix = load_npz(
    MSIGDB_DIRECTORY + "inverse_diag_edge_degree_matrix.npz"
    )

In [4]:
# queue = [[i] for i in range(1,25)]
# resolution = 1.0
# for T in queue:
#     path = f"../output/{DISEASE}/diffusion_dist_matrices/ddm_{T}_res-{resolution}.npy"
#     arr = np.load(path)
#     arr = arr.astype(np.float32)
#     np.save(path, arr)
#     print(f"Overwritten (float32): {path}")

In [5]:
# Useful Functions
def csr_equal_tol(A, B, atol=1e-8):
    # First check shapes and sparsity pattern
    if A.shape != B.shape or not np.array_equal(A.indptr, B.indptr) or not np.array_equal(A.indices, B.indices):
        return False
    # Compare numeric values within tolerance
    return np.allclose(A.data, B.data, atol=atol, rtol=0)

def is_symmetric(W,tol = 1e-8):
    diff = (W - W.T)
    check = np.all(np.abs(diff.data) < tol)
    return check

def degree_array(W, a=1):
    return np.asarray(W.sum(axis=a)).ravel()

def degree_diagonal_matrix(W, a=1):
    d = degree_array(W,a)
    return sp.diags(d, offsets=0, format='csr')

def symmetrically_normalize(W, a=1):
    D = np.asarray(W.sum(axis=a)).ravel()
    D_inv_sqrt = np.zeros_like(D)
    nze = D != 0
    D_inv_sqrt[nze] = 1 / np.sqrt(D[nze])

    W_sym = W.multiply(D_inv_sqrt)              # scale columns
    W_sym = W_sym.multiply(D_inv_sqrt[:, None]) # scale rows    
    return W_sym.tocsr()

def big_objects(n=10, min_mb=1):
    """
    Show the largest objects currently in memory.
    
    Parameters
    ----------
    n : int
        Number of top objects to show.
    min_mb : float
        Minimum size (in MB) to include.
    """
    import sys
    import numpy as np
    import pandas as pd
    import scipy.sparse as sp
    from IPython import get_ipython

    def get_size(obj):
        try:
            if isinstance(obj, np.ndarray):
                return obj.nbytes
            elif isinstance(obj, pd.DataFrame) or isinstance(obj, pd.Series):
                return obj.memory_usage(deep=True).sum()
            elif sp.issparse(obj):
                return (obj.data.nbytes +
                        obj.indptr.nbytes +
                        obj.indices.nbytes)
            else:
                return sys.getsizeof(obj)
        except Exception:
            return 0

    ip = get_ipython()
    if ip is None:
        ns = globals()
    else:
        ns = ip.user_ns

    items = []
    for name, val in ns.items():
        if name.startswith('_'):
            continue  # skip internals
        size = get_size(val)
        if size > min_mb * 1024 ** 2:
            items.append((name, type(val).__name__, size))

    items.sort(key=lambda x: x[2], reverse=True)

    print(f"{'Variable':30s} {'Type':25s} {'Size (MB)':>10s}")
    print("-" * 70)
    for name, t, size in items[:n]:
        print(f"{name:30s} {t:25s} {size / 1024 ** 2:10.2f}")

In [6]:
# Building Adjacency Matrices
## MSIGDB
H,W_v,D_v,D_e_inv = MSIGDB_weighted_matrix, MSIGDB_gene_weight_diag_matrix, MSIGDB_diag_node_degree_matrix, MSIGDB_inverse_diag_edge_degree_matrix

# Construct D_v^(-1/2)
d = (D_v @ W_v).diagonal()
d_inv_sqrt = np.zeros_like(d)
nonzero_mask = d > 0
d_inv_sqrt[nonzero_mask] = 1.0 / np.sqrt(d[nonzero_mask])
D_v_sqrt_inv = diags(d_inv_sqrt)

MSIGDB_adjacency_matrix = D_v_sqrt_inv @ H @ D_e_inv @ H.T @ D_v_sqrt_inv

# Compute Degree Diagonal Matrices
MSIGDB_rows_sums = degree_array(MSIGDB_adjacency_matrix)

In [7]:
print(is_symmetric(MSIGDB_adjacency_matrix))

True


In [8]:
# Symmetric Normalization
MSIGDB_adjacency_matrix = symmetrically_normalize(MSIGDB_adjacency_matrix)

In [9]:
print(is_symmetric(MSIGDB_adjacency_matrix))

True


In [10]:
# Check nonzero average
MSIGDB_nonzero_average = MSIGDB_adjacency_matrix[MSIGDB_adjacency_matrix != 0].mean()

print("\nMSIGDB nonzero average:",MSIGDB_nonzero_average)


MSIGDB nonzero average: 9.87564623806718e-05


In [11]:
## Build the multilayer transition matrix
# interlayer_transition_prob = target_average
multilayer_transition_matrix = MSIGDB_adjacency_matrix
multilayer_transition_matrix = multilayer_transition_matrix.astype(np.float32)

del MSIGDB_adjacency_matrix
gc.collect()

96

In [12]:
multilayer_transition_matrix.shape

(21981, 21981)

In [13]:
num_genes = multilayer_transition_matrix.shape[0]

In [14]:
density = multilayer_transition_matrix.nnz / (multilayer_transition_matrix.shape[0] * multilayer_transition_matrix.shape[1])
print("MTM Density:", density)

MTM Density: 0.22195760212537696


In [15]:
MTM_average = multilayer_transition_matrix[multilayer_transition_matrix != 0].mean()
print("MTM nonzero average:",MTM_average)

MTM nonzero average: 9.8756434e-05


# Computing Eigenvalues & Eigenvectors

In [16]:
num_eigenvalues = 100

In [17]:
# Compute top k eigenvalues by magnitude
def top_k_eigenvalues(M,k):
    vals, vecs = eigsh(M, k=k, which='LM')  # 'LM' = Largest Magnitude
    idx = np.argsort(np.abs(vals))[::-1]
    vals, vecs = vals[idx], vecs[:, idx]
    return vals, vecs

In [18]:
# Compute eigenvalues and eigenvectors
eigenvalues, eigenvectors = top_k_eigenvalues(multilayer_transition_matrix,num_eigenvalues)
print("Eigenvalues:", eigenvalues)
print("Eigenvectors shape:", eigenvectors.shape)

Eigenvalues: [0.99999994 0.9803046  0.88697493 0.8826872  0.86891425 0.8415193
 0.81607425 0.74071735 0.67893374 0.62284887 0.61962944 0.5841457
 0.54950476 0.5028497  0.46663272 0.44824654 0.4351712  0.42835084
 0.41952354 0.41542506 0.41058972 0.40323046 0.39992753 0.39243233
 0.39108595 0.38635355 0.3815382  0.37993115 0.3761189  0.3729739
 0.37221876 0.36879912 0.36657465 0.36528292 0.36145955 0.359357
 0.35685107 0.3554281  0.35042933 0.34965742 0.34251598 0.33906546
 0.33671695 0.33515546 0.33505946 0.33320338 0.33101436 0.32877952
 0.3255512  0.32268518 0.32151964 0.3206363  0.31966653 0.31863508
 0.3165058  0.31466362 0.31385985 0.3092178  0.30721137 0.30606458
 0.30430514 0.30258963 0.3008486  0.29694852 0.2951018  0.29444438
 0.29284498 0.29130796 0.2902719  0.28853112 0.28788957 0.28492132
 0.2828654  0.27918312 0.27772805 0.2761194  0.27515346 0.27272174
 0.2701527  0.2682831  0.2674231  0.26668283 0.26567    0.26441675
 0.2615441  0.25934055 0.25881028 0.25729832 0.2563839

# KNN Graph Methods & Testing (commented)

In [19]:
def stationary_distribution(W, tol=1e-12, maxit=10000):
    # power iteration for left stationary of row-stochastic W
    n = W.shape[0]
    pi = np.ones(n) / n
    for _ in range(maxit):
        pi_next = pi @ W
        if np.linalg.norm(pi_next - pi, 1) < tol:
            break
        pi = pi_next
    return pi / pi.sum()

def layer_weights_stationary(W, n):
    pi = stationary_distribution(W)
    w1 = float(pi[:n].sum()); w2 = 1.0 - w1
    return w1, w2

In [20]:
# Build diffusion matrix from eigenvalues and eigenvectors (double check)
def build_diffusion_dist_matrix(vals,vecs,t):
    X_t = vecs * (vals ** (t))
    D_t2 = squareform(pdist(X_t, metric='sqeuclidean'))
    D_t2 = D_t2.astype(np.float32)
    return D_t2

def build_diffusion_dist_matrix_avg(vals,vecs,T):
    diffusion_dist_matrix = np.zeros((num_genes,num_genes))
    for t in T:
        np.add(diffusion_dist_matrix, 
            build_diffusion_dist_matrix(vals,vecs,t), 
            out=diffusion_dist_matrix)
    diffusion_dist_matrix = diffusion_dist_matrix / len(T)
    return diffusion_dist_matrix

# Build kNN
def build_kNN(diffusion_dist_matrix,k, sym_method='average'):
    # Choose sigma to be the median of pair-wise distance
    tmp = diffusion_dist_matrix.astype(np.float32, copy=True)
    np.sqrt(tmp, out=tmp)
    sigma = np.median(tmp[tmp != 0], overwrite_input=True)

    n = diffusion_dist_matrix.shape[0]

    # Used to indicate position of nonzero value needed to record (constructing a sparse matrix for efficiency)
    rows, cols, vals = [], [], []
    for i in range(n):
        profile = np.exp(-diffusion_dist_matrix[i] / (2* (sigma**2))) 
        idx = np.argpartition(-profile, k+1)[:k+1]  # top-k+1 (includes self)
        idx = idx[idx != i]                      # drop self
        rows += [i]*k
        cols += list(idx[:k])
        vals += list(profile[idx[:k]])
    adj_mat = csr_matrix((vals, (rows, cols)), shape=(n, n))

    # Symmetrize the matrix by adding edges to one way edges and set weight to average
    if (sym_method == 'average'):
        adj_mat = (adj_mat + adj_mat.T).multiply(0.5).tocsr()

    # Symmetrize the matrix by unioning
    elif (sym_method == 'union'):
        adj_mat = adj_mat.maximum(adj_mat.T)
    elif (sym_method == 'none'):
        pass
    else:
        raise ValueError("sym_method must be 'average' or 'union'")
    
    kNN_graph = nx.from_numpy_array(adj_mat)

    
    return adj_mat,kNN_graph

# Clustering Methods

In [21]:
def leiden_from_knn_adjacency(
    A,
    *,
    method="modularity",      # "modularity" | "rb" | "cpm"
    resolution=1.0,           # used by "rb" and "cpm"
    n_iterations=-1,          # -1 => until no improvement
    seed=42,
    use_weights=True
):
    """
    Run Leiden on a symmetric, undirected (weighted) kNN adjacency (SciPy sparse).

    Returns
    -------
    labels : np.ndarray[int]
        Community ID per node (0..k-1)
    quality : float
        Objective value (modularity / RB / CPM depending on 'method')
    """
    if not sp.issparse(A):
        raise TypeError("A must be a SciPy sparse matrix.")
    # Normalize format & dtype
    A = A.tocsr().astype(np.float32, copy=False)

    # Clean diagonal and robustly symmetrize
    A.setdiag(0.0)
    A.eliminate_zeros()
    A = A.maximum(A.T)  # keep max weight per undirected edge

    # Build igraph from upper triangle (each undirected edge once)
    U = sp.triu(A, k=1, format="coo")
    n = A.shape[0]
    g = ig.Graph(n=n, edges=list(zip(U.row.tolist(), U.col.tolist())), directed=False)
    wname = None
    if use_weights:
        g.es["weight"] = U.data.tolist()
        wname = "weight"

    # Pick partition class + kwargs
    m = method.lower()
    if m == "modularity":
        part_cls = la.ModularityVertexPartition
        kwargs = dict(weights=wname)
    elif m == "rb":
        part_cls = la.RBConfigurationVertexPartition
        kwargs = dict(weights=wname, resolution_parameter=resolution)
    elif m == "cpm":
        part_cls = la.CPMVertexPartition
        kwargs = dict(weights=wname, resolution_parameter=resolution)
    else:
        raise ValueError("method must be 'modularity', 'rb', or 'cpm'")

    # Run Leiden
    part = la.find_partition(
        g,
        part_cls,
        n_iterations=n_iterations,
        seed=seed,
        **kwargs
    )

    labels = np.array(part.membership, dtype=np.int32)
    communities = [list(c) for c in part]
    return labels, float(part.quality()), communities

In [22]:
def DDBC(vals,
         vecs,
         num_neighbors,
         resolution,
         T,
         leiden_method = "modularity",
         ddm = None):
    output_folder = f"../output/{DISEASE}/diffusion_dist_matrices"
    os.makedirs(output_folder, exist_ok=True)
    if ddm is None:
        diffusion_dist_matrix = build_diffusion_dist_matrix_avg(vals, vecs,T)
        np.save(f"{output_folder}/ddm_{T}_res-{resolution}_itp-MsigDB.npy",diffusion_dist_matrix)
    else:
        diffusion_dist_matrix = ddm
    # kNN_adjacency_matrix, kNN_graph = build_kNN(diffusion_dist_matrix,num_neighbors)
    kNN_adjacency_matrix, kNN_graph = build_kNN(diffusion_dist_matrix,num_neighbors)

    return (*leiden_from_knn_adjacency(kNN_adjacency_matrix,method=leiden_method,resolution=resolution,n_iterations=-1,seed=42), kNN_graph)

In [23]:
def community_report_onepage(labels, score, out_pdf="leiden_community_report.pdf", *,
                             extra_text=None, bins="auto", title="Community Sizes", time_steps = "N/A"):
    """
    Create ONE PDF page (top: text summary, bottom: histogram).
    If out_pdf exists: append this page. Else: create it.

    Requires: matplotlib, pypdf  (pip install pypdf)
    """
    # ---- inputs ----
    labels = np.asarray(labels)
    if labels.ndim != 1:
        raise ValueError("labels must be a 1-D array of community ids")

    # ---- stats ----
    sizes = np.bincount(labels.astype(np.int64, copy=False))
    sizes_sorted = np.sort(sizes)
    n, k = sizes.sum(), sizes.size

    lines = [
        "Leiden Partition Summary",
        "========================",
        f"Nodes (n):           {n:,}",
        f"Time steps:          {time_steps}",
        f"Communities (k):     {k:,}",
        f"Size (min):          {int(sizes_sorted[0]) if k else 0:,}",
        f"Size (median):       {float(np.median(sizes_sorted)) if k else 0.0:.3f}",
        f"Size (mean):         {float(sizes_sorted.mean()) if k else 0.0:.6f}",
        f"Size (max):          {int(sizes_sorted[-1]) if k else 0:,}",
        f"Score:               {score}",
        "",
        "Top 10 largest communities (id: size):",
    ]
    for cid in np.argsort(-sizes)[:min(10, k)]:
        lines.append(f"  {int(cid):5d}: {int(sizes[cid]):,}")
    if extra_text:
        lines += ["", "Extra:", *([extra_text] if isinstance(extra_text, str) else list(extra_text))]
    summary_text = "\n".join(lines)

    # ---- draw single-page figure ----
    fig = plt.figure(figsize=(8.5, 11), dpi=150)          # US Letter, higher DPI
    ax_text = fig.add_axes([0.06, 0.55, 0.88, 0.40])      # [left, bottom, width, height]
    ax_text.axis("off")
    ax_text.text(0.0, 1.0, summary_text, va="top", ha="left", fontsize=11, family="monospace")

    ax_hist = fig.add_axes([0.10, 0.08, 0.80, 0.38])
    ax_hist.hist(sizes, bins=bins)
    ax_hist.set_xlabel("Community size")
    ax_hist.set_ylabel("Count of communities")
    ax_hist.set_title(f"Distribution of {title}")

    # ---- write this page to a temp PDF on disk ----
    with NamedTemporaryFile(delete=False, suffix=".pdf") as tmpf:
        tmp_page = Path(tmpf.name)
    with PdfPages(tmp_page) as pdf:
        pdf.savefig(fig)
    plt.close(fig)

    # ---- append/create using PdfReader/PdfWriter (robust across versions) ----
    try:
        from pypdf import PdfReader, PdfWriter
    except Exception as e:
        try: os.remove(tmp_page)
        except: pass
        raise RuntimeError("Please install 'pypdf' (e.g., `pip install pypdf`).") from e

    out_pdf = Path(out_pdf)
    tmp_out = out_pdf.with_suffix(out_pdf.suffix + ".tmp")

    writer = PdfWriter()

    # if existing, copy old pages first
    if out_pdf.exists():
        with open(out_pdf, "rb") as f_exist:
            reader = PdfReader(f_exist)
            for p in reader.pages:
                writer.add_page(p)

    # add the new single page
    with open(tmp_page, "rb") as f_new:
        reader_new = PdfReader(f_new)
        for p in reader_new.pages:
            writer.add_page(p)

    # atomic write
    with open(tmp_out, "wb") as f_out:
        writer.write(f_out)
    os.replace(tmp_out, out_pdf)

    # cleanup
    try: os.remove(tmp_page)
    except: pass

    return out_pdf


# Run DDBC 1 - 24

In [ ]:
# Leiden Clustering
pdf_path = f"../output/{DISEASE}/leiden_report.pdf"
method = "modularity"
resolution = 1.3
num_neighbors = 100
num_bins = 100
write = True
queue = [[i] for i in range(1,2)]
label_list = []
score_list = []
graph_list = []
communities_list = []

In [ ]:
# # Load variables:
# DATA_DIRECTORY = OUTPUT_DIRECTORY + "leiden_result_variables_temp"
# with open(f"{DATA_DIRECTORY}/label.pkl", "rb") as f:
#     label_list = pickle.load(f)
# with open(f"{DATA_DIRECTORY}/score.pkl", "rb") as f:
#     score_list = pickle.load(f)
# with open(f"{DATA_DIRECTORY}/graph.pkl", "rb") as f:
#     graph_list = pickle.load(f)
# with open(f"{DATA_DIRECTORY}/communities.pkl", "rb") as f:
#     communities_list = pickle.load(f)

In [ ]:
for T in queue:
    labels, score, communities, graph = DDBC(eigenvalues, eigenvectors, num_neighbors, resolution, T, leiden_method = method)
    if (write):
        path = community_report_onepage(labels, score, out_pdf=pdf_path,
                                        extra_text=[f"method={method}", f"resolution={resolution}"], 
                                        bins=num_bins,time_steps = T)
        print("Wrote:", path)
    label_list.append(labels)
    score_list.append(score) 
    graph_list.append(graph)
    communities_list.append(communities)

In [ ]:
# # Save resulting variables to save time
# DATA_DIRECTORY = OUTPUT_DIRECTORY + "leiden_result_variables_temp"
# with open(f"{DATA_DIRECTORY}/label.pkl", "wb") as f:
#     pickle.dump(label_list, f)
# with open(f"{DATA_DIRECTORY}/score.pkl", "wb") as f:
#     pickle.dump(score_list, f)
# with open(f"{DATA_DIRECTORY}/graph.pkl", "wb") as f:
#     pickle.dump(graph_list, f)
# with open(f"{DATA_DIRECTORY}/communities.pkl", "wb") as f:
#     pickle.dump(communities_list, f)

# Visualization

In [ ]:
def community_central_genes(G, community_nodes, weight="weight", top_n=20):
    C = set(community_nodes)
    H = G.subgraph(C).copy()                       # induced subgraph
    # within-community (weighted) degree
    k = {u: H.degree(u, weight=weight) for u in H}
    ks = np.array(list(k.values()), dtype=float)
    mu, sigma = ks.mean(), ks.std() if ks.std() > 0 else 1.0
    Z = {u: (k[u] - mu)/sigma for u in H}          # within-module degree z-score

    # rank by z
    ranked = sorted(H.nodes(), key=lambda u: (Z[u]), reverse=True)
    return {u : Z[u] for u in ranked[:top_n]}

def weighted_jaccard(scoresA, scoresB):
    """
    Compute Weighted Jaccard similarity between two communities
    based on gene importance scores.
    
    Parameters:
        scoresA, scoresB : dict
            {gene: importance_score}
            Scores can be any nonnegative values (e.g., PageRank, Z-score).
    Returns:
        float
            Weighted Jaccard similarity in [0, 1].
    """
    genes = set(scoresA) | set(scoresB)
    if not genes:
        return 0.0
    num = sum(min(scoresA.get(g, 0.0), scoresB.get(g, 0.0)) for g in genes)
    den = sum(max(scoresA.get(g, 0.0), scoresB.get(g, 0.0)) for g in genes)
    return num / den if den > 0 else 0.0

def weighted_overlap_coefficient(dictA, dictB):
    """
    Weighted Szymkiewicz–Simpson (Overlap) coefficient ∈ [0,1].
    """
    if not dictA or not dictB:
        return 0.0

    common = set(dictA) & set(dictB)
    inter_sum = sum(min(dictA[v], dictB[v]) for v in common)

    sumA = sum(max(0, w) for w in dictA.values())
    sumB = sum(max(0, w) for w in dictB.values())
    denom = min(sumA, sumB)

    return inter_sum / denom if denom > 0 else 0.0




In [ ]:
def wjacc_edges_builder(g_list,c_list,score_cap = 0,com_size_cap = 0):
    result = []
    for i in range(len(g_list)-1):
        prev_G = g_list[i]
        curr_G = g_list[i+1]
        prev_com = c_list[i]
        curr_com = c_list[i+1]
        prev_z = []
        curr_z = []
        
        for com in prev_com:
            if (len(com) > com_size_cap):
                prev_z.append(community_central_genes(prev_G,com))
        for com in curr_com:
            if (len(com) > com_size_cap):
                curr_z.append(community_central_genes(curr_G,com))

        for j in range(len(prev_z)):
            for k in range(len(curr_z)):
                jaccard_score = weighted_jaccard(prev_z[j],curr_z[k])
                if (jaccard_score >= score_cap):
                    result.append((queue[i][0],j,k,jaccard_score))
    
    return result

def community_sizes(labels):
    """Return dict[label] -> number of nodes in that community."""
    lab = np.asarray(labels)
    uniq, cnt = np.unique(lab, return_counts=True)
    return {int(c): int(n) for c, n in zip(uniq, cnt)}

def topk_at_t1(labels_t1, k=10):
    """Return list of top-k community labels at t=1 by size."""
    sizes = community_sizes(labels_t1)
    return [c for c, _ in sorted(sizes.items(), key=lambda x: x[1], reverse=True)[:k]], sizes

# ---------- Jaccard lookup ----------

def build_edge_lookup(wjacc_edges):
    """
    Build a dict mapping (t, comm_t) -> list of (comm_t+1, jaccard_score).
    Each wjacc_edges element = (t, comm_t, comm_tplus1, jaccard_score)
    """
    out = defaultdict(list)
    for t, c1, c2, s in wjacc_edges:
        out[(int(t), int(c1))].append((int(c2), float(s)))
    return out

# ---------- Tracking ----------

def track_paths(seeds, wj_lookup, ts):
    """Track each seed community forward by max Jaccard each step."""
    print(ts)
    t0 = ts[0]
    paths = {s: [(t0, s)] for s in seeds}
    edges = []
    for s in seeds:
        cur = s
        for i in range(len(ts) - 1):
            t = ts[i]
            tp1 = ts[i+1]
            cand = wj_lookup.get((t, cur), [])
            if not cand:
                print(f"Seed {s}, {t} to {tp1} has no outgoing edges. Termniated.")
                paths[s].append((tp1, None))
                cur = None
                break
            nxt, score = max(cand, key=lambda x: x[1])
            edges.append(((t, cur), (tp1, nxt), score))
            paths[s].append((tp1, nxt))
            cur = nxt
    return paths, edges

# MY VERSION
# def track_paths(labels_by_t, seeds, wj_lookup, ts,score_cap = 0.1):
#     """Track each seed community forward by max Jaccard each step."""
#     t0 = ts[0]
#     paths = {s: [(t0, s)] for s in seeds}
#     edges = []
#     curs = []
#     curs_next = []
#     for s in seeds:
#         curs_next.append(s)
#         for i in range(len(ts) - 1):
#             curs = curs_next
#             curs_next = []
#             for cur in curs:
#                 t = ts[i]
#                 tp1 = ts[i+1]
#                 cand = wj_lookup.get((t, cur), [])
#                 if not cand:
#                     if (t == t0):
#                         print(f"Community {cur} has no outgoing edges")
#                     paths[s].append((tp1, None))
#                     cur = None
#                     continue
                
                
#                 for nxt,score in cand:
#                     if (score >= score_cap):
#                         edges.append(((t, cur), (tp1, nxt), score))
#                         paths[s].append((tp1, nxt))
#                         curs_next.append(nxt)

#     return paths, edges

# ---------- Get node sizes ----------

def node_sizes_for_paths(labels_by_t, paths):
    """Return dict[(t, comm)] -> size (number of vertices)."""
    out = {}
    for s, seq in paths.items():
        for t, c in seq:
            if c is None: continue
            sizes = community_sizes(labels_by_t[t])
            out[(t, c)] = sizes.get(c, 0)
    return out

# ---------- Draw layered network ----------

def draw_layered_paths(paths, edge_list, node_sizes, ts,
                       node_size_scale=700.0, edge_width_scale=8.0,
                       seed_gap=1.0, col_gap=3.0, score_cap = 0.5, title=None, save_path = None):
    seeds = list(paths.keys())
    G = nx.DiGraph()

    # Add nodes
    for s in seeds:
        for t, c in paths[s]:
            if c is None: continue
            G.add_node((s, t, c), seed=s, t=t, comm=c, size=node_sizes.get((t, c), 0))
    # Add edges
    # Set your inclusion threshold (cap)
    # include all edges with Jaccard >= 0.5

    for (t, c1), (tp, c2), w in edge_list:
        if w < score_cap:
            continue
        for s in seeds:
            seq = paths[s]
            for i in range(len(seq) - 1):
                if seq[i] == (t, c1) and seq[i+1] == (tp, c2):
                    G.add_edge((s, t, c1), (s, tp, c2), jacc=w)

    # Layout positions
    pos = {}
    nodes_by_t = defaultdict(list)
    for n in G.nodes:
        t = G.nodes[n]['t']
        nodes_by_t[t].append(n)

    # Sort nodes in each column by community size (largest first)
    pos = {}
    for t in ts:
        column_nodes = nodes_by_t.get(t, [])
        # sort by size descending
        column_nodes.sort(key=lambda n: G.nodes[n].get('size', G.nodes[n].get('size_w', 0)), reverse=True)
        for i, n in enumerate(column_nodes):
            pos[n] = (ts.index(t) * col_gap, -i * seed_gap)

    # Scale sizes and widths
    if len(G) == 0:
        raise ValueError("No nodes to draw — check your data.")
    vals = np.array([G.nodes[n]['size'] for n in G.nodes], dtype=float)
    vmin, vmax = vals.min(), vals.max()
    sizes = []
    for n in G.nodes:
        w = G.nodes[n]['size']
        a = (w - vmin) / (vmax - vmin) if vmax > vmin else 1.0
        sizes.append((0.3 + 0.7*a) * node_size_scale)
    widths = [max(0.5, d['jacc'] * edge_width_scale) for _, _, d in G.edges(data=True)]

    # Draw
    plt.figure(figsize=(max(10, len(ts)*1.4), max(6, len(seeds)*0.55 + 2)))
    nx.draw_networkx_nodes(G, pos, node_size=sizes)
    nx.draw_networkx_edges(G, pos, width=widths, arrows=False)

    edge_labels = {(u, v): f"{d['jacc']:.2f}" for u, v, d in G.edges(data=True)}
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8)
    
    plt.xticks([i * col_gap for i in range(len(ts))], [str(t) for t in ts])
    plt.yticks([])
    plt.title(title or f"Top {len(seeds)} communities from t={ts[0]} → t={ts[-1]} (edge ∝ Jaccard)")
    plt.tight_layout()
    if (save_path != None):
        plt.savefig(save_path, dpi=300, bbox_inches="tight")
    plt.show()

# ---------- Main wrapper ----------

def visualize_topk_from_edges(labels_by_t, wjacc_edges,
                              t_start=1, t_end=16, top_k=10,
                              node_size_scale=700.0, score_cap = 0.5,
                              edge_width_scale=8.0, title=None, save_path = None):
    ts = sorted(labels_by_t.keys())
    if t_start not in labels_by_t:
        raise ValueError(f"Missing labels for t={t_start}")

    # Top-k seeds from t_start
    seeds, _ = topk_at_t1(labels_by_t[t_start], k=top_k)

    # Track forward using provided Jaccard edges
    wj_lookup = build_edge_lookup(wjacc_edges)
    paths, edges = track_paths(seeds, wj_lookup, ts)

    # Compute community sizes (unweighted)
    node_sizes = node_sizes_for_paths(labels_by_t, paths)

    # Draw the layered visualization
    print(ts)
    draw_layered_paths(paths, edges, node_sizes, ts,
                       node_size_scale=node_size_scale,
                       edge_width_scale=edge_width_scale,
                       title=title or f"Top-{top_k} from t={t_start} → t={t_end}", score_cap = score_cap,
                       save_path = save_path)

    return paths, edges


In [ ]:
# # Load variables:
# DATA_DIRECTORY = OUTPUT_DIRECTORY + "leiden_result_variables_temp"
# with open(f"{DATA_DIRECTORY}/label.pkl", "rb") as f:
#     label_list = pickle.load(f)
# with open(f"{DATA_DIRECTORY}/score.pkl", "rb") as f:
#     score_list = pickle.load(f)
# with open(f"{DATA_DIRECTORY}/graph.pkl", "rb") as f:
#     graph_list = pickle.load(f)
# with open(f"{DATA_DIRECTORY}/communities.pkl", "rb") as f:
#     communities_list = pickle.load(f)

In [ ]:
print(len(graph_list))
print(len(communities_list))

In [ ]:
labels_dict = {queue[i][0]: label_list[i] for i in range(len(queue))}
wjacc = wjacc_edges_builder(graph_list,communities_list)

In [ ]:
print(labels_dict)
print(len(labels_dict))

In [ ]:
print(wjacc)
print(len(wjacc))
save_path = f"../output/{DISEASE}/leiden_tracking_results/leiden_tracking.png"
# save_path = None
_, edges = visualize_topk_from_edges(labels_dict, wjacc, 1,24,top_k = 20,score_cap = 0.03,save_path=save_path)

In [ ]:
print(score_list)

In [ ]:
queue = [[i] for i in range(1,25)]
x = [T[0] for T in queue]
y1 = score_list
y2 = [len(communities_list[i]) for i in range(len(communities_list))]

# Create a figure with 2 subplots (1 row, 2 columns)
fig, axes = plt.subplots(1, 2, figsize=(10, 4))

# Left graph
axes[0].plot(x, y1, color='r')
axes[0].set_title("quality score")
axes[0].set_xlabel("x")
axes[0].set_ylabel("quality_score")
axes[0].grid(True)

# Right graph
axes[1].plot(x, y2, color='b')
axes[1].set_title("community size")
axes[1].set_xlabel("x")
axes[1].set_ylabel("community size")
axes[1].grid(True)

# Adjust spacing
plt.tight_layout()
plt.show()

In [ ]:
mixed_time = 1 / (1-eigenvalues[1])
print(f"Mixed Time: {mixed_time}")

# Final Average DDBC

In [24]:
# Final Average Clustering
average_t = [2,4,6,8,10,12]
pdf_path = f"../output/{DISEASE}/leiden_report.pdf"
method = "rb"
num_neighbors = 200
resolution = 1.3
label_list = []
score_list = []
graph_list = []
communities_list = []

In [25]:
print(multilayer_transition_matrix)

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 107242003 stored elements and shape (21981, 21981)>
  Coords	Values
  (0, 0)	0.0012642317451536655
  (0, 1)	3.1965315429260954e-05
  (0, 2)	1.4127141184872016e-05
  (0, 4)	2.456477432133397e-06
  (0, 5)	2.35497864196077e-05
  (0, 7)	6.282188223849516e-06
  (0, 10)	1.7112533896579407e-05
  (0, 17)	1.3418345588434022e-05
  (0, 18)	1.8281893062521704e-05
  (0, 22)	3.386093521839939e-05
  (0, 24)	0.00037356969551183283
  (0, 25)	9.248384230886586e-06
  (0, 26)	6.520564966194797e-06
  (0, 27)	7.4472545747994445e-06
  (0, 30)	7.925462341518141e-06
  (0, 35)	8.841525414027274e-06
  (0, 36)	6.497576123365434e-06
  (0, 39)	8.956683814176358e-06
  (0, 43)	2.1123141777934507e-05
  (0, 47)	4.650702521757921e-06
  (0, 48)	7.167964213294908e-06
  (0, 50)	2.5968322461267235e-06
  (0, 52)	4.737279596156441e-05
  (0, 55)	2.590298026916571e-05
  (0, 56)	5.90161798754707e-05
  :	:
  (21980, 16835)	5.565654646488838e-05
  (21980, 16978)	0.00013

In [26]:
labels, score, communities, graph = DDBC(eigenvalues, eigenvectors, num_neighbors, resolution, average_t, leiden_method = method)
path = community_report_onepage(labels, score, out_pdf=pdf_path,
                                extra_text=[f"method={method}", f"resolution={resolution}", f"number of neighbors: {num_neighbors}"], 
                                bins=100,time_steps = average_t)
print("Wrote:", path)

Wrote: ..\output\NONE\leiden_report.pdf


In [27]:
print(communities)
DATA_DIRECTORY = OUTPUT_DIRECTORY + "leiden_results"
os.makedirs(DATA_DIRECTORY, exist_ok=True)
with open(f"{DATA_DIRECTORY}/result_communities.pkl", "wb") as f:
    pickle.dump(communities, f)
with open(f"{DATA_DIRECTORY}/result_graph.pkl", "wb") as f:
    pickle.dump(graph, f)    

[[10, 21, 25, 26, 28, 33, 35, 43, 87, 89, 91, 92, 96, 99, 104, 105, 107, 109, 111, 112, 113, 114, 120, 135, 136, 138, 139, 141, 144, 145, 149, 150, 152, 153, 154, 155, 157, 169, 171, 174, 185, 208, 221, 222, 226, 235, 250, 253, 258, 266, 269, 272, 279, 285, 292, 295, 296, 297, 299, 300, 305, 320, 330, 337, 339, 344, 345, 346, 350, 356, 363, 375, 376, 378, 384, 396, 402, 406, 407, 411, 416, 417, 419, 420, 432, 440, 447, 451, 454, 463, 470, 471, 478, 479, 494, 495, 499, 501, 511, 518, 519, 522, 529, 538, 540, 549, 561, 562, 566, 577, 586, 596, 603, 606, 614, 615, 616, 621, 622, 633, 634, 635, 640, 647, 649, 659, 663, 664, 669, 675, 676, 678, 681, 683, 685, 687, 691, 703, 711, 726, 730, 748, 769, 771, 776, 777, 793, 794, 810, 812, 815, 820, 821, 829, 832, 833, 834, 835, 838, 839, 841, 846, 865, 873, 875, 876, 878, 887, 889, 892, 893, 896, 911, 925, 945, 974, 982, 983, 985, 986, 992, 998, 1000, 1002, 1007, 1048, 1049, 1061, 1066, 1070, 1073, 1077, 1085, 1088, 1093, 1114, 1116, 1119, 1121, 

### Create stub files (for MsigDB)

In [28]:
if (DISEASE == "NONE"):
    GEN_HYPERGRAPH = "../../Gen_Hypergraph/output"
    src = f"{GEN_HYPERGRAPH}/MsigDB_FULL/gene_to_index.json"
    dst = "../output/NONE/gene_to_index_distinct.json"  # or folder path only
    shutil.copy(src, dst)
    
    DST = f"{GEN_HYPERGRAPH}/DGIDB_{DISEASE}"
    os.makedirs(DST,exist_ok = True)
    empty_dics = [f"gene_to_index.json",f"drug_to_index.json"]
    for name in empty_dics:
        path = os.path.join(DST, name)
        with open(path, "w") as f:
            json.dump({}, f)   # write empty dict

# Robustness Analysis

In [ ]:
ddm_test = build_diffusion_dist_matrix_avg(eigenvalues, eigenvectors,[2,4,6,8,10,12])

In [ ]:
ddm_test = ddm_test.astype(np.float32, copy = False)

In [ ]:
del multilayer_transition_matrix
gc.collect()

In [ ]:
similarity_matrix,_ = build_aggregated_kNN(ddm_test, k = 200)

In [ ]:
labels_original,_,_ = leiden_from_knn_adjacency(similarity_matrix, method="modularity",resolution=1.0,n_iterations=-1,seed=42)

In [ ]:
n_boots = 50
ari_scores = []

for b in tqdm(range(n_boots)):
    # 1. Sample genes with replacement
    n = similarity_matrix.shape[0]
    idx = np.random.choice(np.arange(n), size=n, replace=True)
    sim_resampled = similarity_matrix[idx, :][:, idx]
    
    # 2. Recompute similarity and cluster
    labels_b, _, _ = leiden_from_knn_adjacency(sim_resampled, method="modularity",resolution=1.0,n_iterations=-1,seed=42)
    
    # 3. Compare with original clustering
    ari = adjusted_rand_score(labels_original[idx], labels_b)
    ari_scores.append(ari)

print("Median ARI:", np.median(ari_scores))